In [5]:
モデルを試したスコア 特徴量は以下の通りで一致させた　
特徴量について試行錯誤した人がいたら共有してもらいたい
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]
Index(['client', 'close', 'price_am', 'price_pm', 'year_num', 'M_1', 'M_2',
       'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12',
       'DM_1', 'DM_2', 'DM_3', 'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9',
       'DM_10', 'DM_11', 'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17',
       'DM_18', 'DM_19', 'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25',
       'DM_26', 'DM_27', 'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1',
       'DW_2', 'DW_3', 'DW_4', 'DW_5', 'DW_6'],
      dtype='object')

重回帰分析　pred_clf
11.376637316800000

xgboostで回帰モデル生成　pred_xgb
mod = xgb.XGBRegressor() パラメーターはいじらず初期設定のまま
11.17721

ランダムフォレスト forest_pred
from sklearn.ensemble import RandomForestRegressor
モデル構築、パラメータはデフォルト
forest = RandomForestRegressor()
11.48777

サポートベクターマシーン
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=3)
y_rbf = svr_rbf.fit(X, Y).predict((test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))
y_lin = svr_lin.fit(X, Y).predict((test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))
y_poly = svr_poly.fit(X, Y).predict((test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))
y_poly 12.50321
y_rbf #10.924218922800000

Kerasによる回帰分析
estimator = KerasRegressor(build_fn=deep_reg_model, nb_epoch=100, batch_size=10, verbose=0)
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]
X = np.array(X)
Y = np.array(Y)
estimator.fit(X,Y)
12.17040

y_rbfとpred_xgbも平均
test_df["y"] = (y_rbf + pred_xgb)/2
10.22758

test_df["y"] = (y_rbf + pred_xgb+forest_pred)/3
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
10.29414

In [1]:
#モデルを試して行くぜ
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt



%matplotlib inline
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)
train_df_y.reset_index()
df = pd.concat([train_df,test_df])
df['datetime'] = pd.to_datetime(df.datetime)
#何月か
df['Month'] = df['datetime'].dt.month
#何日か
df['DayofMonth'] = df['datetime'].dt.day
#何曜日か
df['dayofweek'] = df['datetime'].dt.dayofweek
#何年か
df['year'] = df['datetime'].dt.year
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df["year"])

df["year_num"] = le.transform(df["year"])
#ダミー変数化
#何月か
month_df = pd.get_dummies(df['Month'], prefix='M', prefix_sep='_')
#何日か
dayofmonth_df = pd.get_dummies(df["DayofMonth"], prefix='DM', prefix_sep='_')
#何曜日か
dayofweek_df = pd.get_dummies(df['dayofweek'], prefix='DW', prefix_sep='_')
#何年か
#year_df= pd.get_dummies(df['year'], prefix='y', prefix_sep='_')
#datetimeを保存
df_id = df[['datetime']]
#year_df削除
df = pd.concat([df,
                month_df,
                dayofmonth_df,
                dayofweek_df], axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [2]:
#重回帰分析で予測
from sklearn import linear_model
clf = linear_model.LinearRegression()
 


X = train_df.drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
 

Y = train_df["y"]
 
# 予測モデルを作成
clf.fit(X, Y)
pred = clf.predict(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1))

In [3]:
#重回帰分析で予測
from sklearn import linear_model
clf = linear_model.LinearRegression()
 
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]


 
# 予測モデルを作成
clf.fit(X, Y)
pred_clf = clf.predict(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1))
#11.376637316800000

In [4]:
X.columns

Index(['client', 'close', 'price_am', 'price_pm', 'year_num', 'M_1', 'M_2',
       'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12',
       'DM_1', 'DM_2', 'DM_3', 'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9',
       'DM_10', 'DM_11', 'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17',
       'DM_18', 'DM_19', 'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25',
       'DM_26', 'DM_27', 'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1',
       'DW_2', 'DW_3', 'DW_4', 'DW_5', 'DW_6'],
      dtype='object')

In [96]:
test_df["y"] = pred_clf
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
#11.376637316800000

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  se

In [72]:
#xgboostで回帰モデル生成
import xgboost as xgb
#モデルのインスタンス作成
mod = xgb.XGBRegressor()
#欠損値のデータを学習させずにモデル生成
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]
mod.fit(X,Y)
pred_xgb= mod.predict(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1))
#11.17721

In [4]:
# test_df["y"] = pred_xgb
# test_df["y"][test_df["close"]==1] = int(0)
# submission = pd.concat([test_df.datetime, test_df.y], axis=1)
# submission.to_csv("submission.csv", index=False,header=False)
#11.17721

In [5]:
#ランダムフォレスト
from sklearn.cross_validation import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(X, Y, test_size = 0.3, random_state = 666)
from sklearn.ensemble import RandomForestRegressor
# モデル構築、パラメータはデフォルト
forest = RandomForestRegressor()
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)
# MSEの計算
from sklearn.metrics import mean_squared_error
print('MSE train : %.3f, test : %.3f' % (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)) )


MSE train : 13.729, test : 85.372


In [73]:
from sklearn.ensemble import RandomForestRegressor
# モデル構築、パラメータはデフォルト
forest = RandomForestRegressor()
forest.fit(X,Y)
#11.48777

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [74]:
forest_pred = forest.predict(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1))
# test_df["y"] = forest_pred
# test_df["y"][test_df["close"]==1] = int(0)
# submission = pd.concat([test_df.datetime, test_df.y], axis=1)
# submission.to_csv("submission.csv", index=False,header=False)
#11.48777

In [75]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=3)
y_rbf = svr_rbf.fit(X, Y).predict((test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))
y_lin = svr_lin.fit(X, Y).predict((test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))
y_poly = svr_poly.fit(X, Y).predict((test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))

In [9]:
test_df

,datetime,client,close,price_am,price_pm,Month,DayofMonth,dayofweek,year,year_num,...,DM_29,DM_30,DM_31,DW_0,DW_1,DW_2,DW_3,DW_4,DW_5,DW_6
0,2016-04-01,1,0,3,2,4,1,4,2016,6,...,0,0,0,0,0,0,0,1,0,0
1,2016-04-02,0,0,5,5,4,2,5,2016,6,...,0,0,0,0,0,0,0,0,1,0
2,2016-04-03,1,0,2,2,4,3,6,2016,6,...,0,0,0,0,0,0,0,0,0,1
3,2016-04-04,1,0,1,1,4,4,0,2016,6,...,0,0,0,1,0,0,0,0,0,0
4,2016-04-05,0,0,1,1,4,5,1,2016,6,...,0,0,0,0,1,0,0,0,0,0
5,2016-04-06,1,0,1,1,4,6,2,2016,6,...,0,0,0,0,0,1,0,0,0,0
6,2016-04-07,1,0,1,1,4,7,3,2016,6,...,0,0,0,0,0,0,1,0,0,0
7,2016-04-08,0,0,1,1,4,8,4,2016,6,...,0,0,0,0,0,0,0,1,0,0
8,2016-04-09,1,0,2,2,4,9,5,2016,6,...,0,0,0,0,0,0,0,0,1,0
9,2016-04-10,1,0,1,1,4,10,6,2016,6,...,0,0,0,0,0,0,0,0,0,1


In [10]:
y_rbf

array([  78.93042187,   69.79739589,   85.43182115,   73.38358769,
         64.12750312,   64.82967146,   67.95463244,   63.20323627,
         72.63644837,   80.32479388,   52.31645745,   54.12457331,
         61.92941689,   45.39051669,   55.73722672,   66.31211035,
         64.76027015,   44.48929169,   37.22134208,   49.91748989,
         51.34421695,   66.33099451,   70.69915676,   64.12646831,
         74.05863524,   52.71128553,   71.22005793,   79.94037724,
         94.28191127,   88.81936482,   68.62558827,   53.53993471,
         60.1761598 ,   61.9154712 ,   52.45082356,   48.36767278,
         64.21170247,   46.22978622,   46.25174848,   43.25133887,
         44.39915884,   40.36130463,   50.93447062,   70.82418669,
         55.35688329,   45.21448768,   43.01315291,   53.64750762,
         45.09544376,   78.6139185 ,   84.0017609 ,   69.12541784,
         61.21213736,   52.78040875,   52.35766655,   52.55340427,
         70.29438532,   75.34410571,   77.59806744,   70.65316

In [11]:
y_lin

array([  65.28553148,   79.17664002,   59.08285445,   51.53181088,
         49.99922863,   46.10909298,   47.16186931,   52.97463416,
         57.20953475,   55.46168653,   46.10322364,   44.07261799,
         45.95607914,   46.24094145,   55.12535744,   62.28476455,
         57.89092391,   51.93113748,   48.68657057,   51.37920717,
         51.9103924 ,   60.01529467,   70.10310488,   65.1544171 ,
         62.93046328,   56.35232426,   59.86721081,   62.79532969,
         75.95372416,   77.0465495 ,   57.57295736,   43.05780416,
         43.66166801,   43.28866   ,   38.43692962,   41.45221603,
         46.66391238,   43.49625755,   39.54825879,   41.16789515,
         37.86007276,   39.62723813,   49.74664537,   49.96670611,
         48.74330736,   45.75088365,   40.70393497,   40.79478903,
         41.34799315,   52.27657584,   58.73248359,   53.63324459,
         49.54863136,   47.76429919,   47.57039324,   49.01374685,
         57.66825295,   66.52109435,   65.34795248,   57.61947

In [12]:
y_poly

array([  69.89918856,   60.65476818,   67.14713023,   63.38540259,
         48.56704305,   46.49016771,   47.30516658,   44.63111907,
         71.45785734,   66.86666044,   50.79599946,   48.0978724 ,
         53.39690875,   46.97644757,   51.60573904,   67.68202105,
         52.96902196,   48.04066144,   37.97397817,   49.92179223,
         44.53641901,   60.29446775,   67.10250065,   70.973678  ,
         67.35757247,   56.69501675,   61.18297048,   65.80349196,
         80.91899296,   82.24255069,   43.69134808,   33.12833289,
         26.82400599,   22.85770254,   20.24315465,   23.9219684 ,
         33.09821591,   22.52297686,   21.64039611,   24.21016607,
         20.00873182,   25.35533686,   28.33816632,   42.22876131,
         31.28794192,   33.09832271,   22.67989791,   28.29795466,
         26.94661616,   39.12576001,   49.30630985,   41.37068777,
         40.93737452,   35.2958596 ,   40.76679814,   38.80108833,
         46.10064374,   52.92000747,   58.57458666,   52.05363

In [13]:
# test_df["y"] = y_poly
# test_df["y"][test_df["close"]==1] = int(0)
# submission = pd.concat([test_df.datetime, test_df.y], axis=1)
# submission.to_csv("submission.csv", index=False,header=False)
# #12.50321

In [14]:
#submission

In [15]:
test_df["y"] = y_rbf
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
#10.924218922800000

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  se

In [2]:
# import tensorflow as tf
# import numpy as np
# import pandas as pd
# from sklearn.datasets import load_boston
# from sklearn.preprocessing import StandardScaler

In [3]:
#ボストンデータ読み込み
# boston = load_boston()
# df = pd.DataFrame(boston.data, columns = boston.feature_names)
# df['target'] = boston.target

#学習データ準備
#f_num = df.shape[1] - 1
# X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
# Y = train_df[train_df["price_pm"]>-1]["y"]
# train_X = np.array(X)
# train_Y = np.array(Y)
# n_samples = train_X.shape[0]
# f_num = X.shape[1]

# #正規化
# ss = StandardScaler()
# ss.fit(train_X)
# train_X = ss.transform(train_X)

/Users/haradakeno/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [10]:
# #プレースホルダー
# with tf.name_scope('data'):
#     x = tf.placeholder(tf.float32, shape = [None, f_num ], name = "x")
#     y = tf.placeholder(tf.float32, name = "y")

# #変数（パラメータ）
# with tf.name_scope('parameter'):
#     W = tf.Variable(tf.zeros([f_num, 1]), name = "weight")
#     b = tf.Variable(tf.zeros([1]), name = "bias")

In [13]:
# with tf.name_scope('model'):
#     pred = tf.add(tf.matmul(x, W), b) #matmulは行列の積

In [15]:
# with tf.name_scope('loss'):
#     # Mean squared error
#     loss = tf.reduce_mean(tf.square(pred - y))
#     tf.summary.scalar('loss', loss)

# optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
# train_step = optimizer.minimize(loss)

# #決定係数(R2)
# with tf.name_scope('r2'):
#     r2 = 1 - (tf.reduce_sum(tf.square(y - pred)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y))))
#     tf.summary.scalar('r2', r2)

In [34]:
# with tf.Session() as sess:
#     # ログの設定
#     summary = tf.summary.merge_all()
#     writer = tf.summary.FileWriter("boston_log", sess.graph)
    
#     sess.run(tf.global_variables_initializer())#変数初期化
    
#     for i in range(1000):
#         sess.run(train_step, feed_dict={x: train_X, y: train_Y})
#         if i != 0 and i % 200 == 0: # 200ステップごとに精度を出力
#             train_summary, train_loss, train_r2 = sess.run([summary, loss, r2], feed_dict={x: train_X, y:train_Y})# コストと精度を出力
#             writer.add_summary(train_summary, i) #summaryの更新
            
#             print("Step:", '%04d' % (i), "loss=", "{:.9f}".format(train_loss), "r2=", "{:.9f}".format(train_r2), "W=", sess.run(W), "b=", sess.run(b))            
            
#     training_cost, training_r2 = sess.run([loss,r2], feed_dict={x: train_X, y: train_Y})
#     print("Training cost=", training_cost, "Training r2=", training_r2, "W=", sess.run(W), "b=", sess.run(b), '\n')
#     W_f = sess.run(W)
#     b_f = sess.run(b)

Step: 0200 loss= 284.062591553 r2= -1890.659057617 W= [[  4.94321881e-08]
 [  0.00000000e+00]
 [  1.95282013e-07]
 [  9.92184255e-08]
 [  1.67237829e-07]
 [ -1.88804933e-07]
 [ -2.77881867e-07]
 [ -3.42442746e-07]
 [ -4.63631778e-07]
 [ -2.79550335e-07]
 [ -4.00711798e-07]
 [ -2.54025508e-07]
 [ -3.52539672e-07]
 [ -4.11924702e-07]
 [ -2.93066478e-07]
 [ -4.15054046e-07]
 [ -4.85350142e-07]
 [ -1.77847795e-07]
 [  1.26929606e-07]
 [ -1.46215498e-07]
 [  2.59804178e-08]
 [  2.90863724e-08]
 [  3.26789369e-08]
 [  3.31306360e-08]
 [  4.33984617e-08]
 [  4.49514346e-08]
 [  3.64996922e-08]
 [  3.50910625e-08]
 [ -9.32195476e-09]
 [  4.94869923e-08]
 [  3.10328225e-08]
 [  3.16055839e-08]
 [  3.51236586e-08]
 [  3.16102380e-08]
 [  2.80805423e-08]
 [  2.63436224e-08]
 [  2.23668675e-08]
 [  2.42877185e-08]
 [  2.33051765e-08]
 [  1.24715793e-08]
 [  1.12375531e-08]
 [ -1.13273013e-09]
 [ -3.81959131e-09]
 [  2.05240647e-09]
 [ -1.89175031e-09]
 [  1.21139578e-08]
 [  6.82807482e-08]
 [ -1.

In [36]:
#pred_tf= np.dot(np.array(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)),W_f) +b_f

In [25]:
#test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1).shape

(365, 55)

In [39]:
#pred_tf

In [ ]:
#Kerasで回帰分析

In [41]:
import numpy as np
import pandas as pds
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_diabetes

# use diabetes sample data from sklearn


# load them to X and Y
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]

In [83]:
# create regression model
def reg_model():
    model = Sequential()
    model.add(Dense(10, input_dim=55, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    # compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [89]:
def deep_reg_model():
    model = Sequential()
    model.add(Dense(10, input_dim=55, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1))

    # compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(build_fn=deep_reg_model, nb_epoch=100, batch_size=10, verbose=0)
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]
X = np.array(X)
Y = np.array(Y)
estimator.fit(X,Y)

In [ ]:
test_df["y"] = estimator.predict(np.array(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
#12.17040

In [80]:
test_df["y"] = estimator.predict(np.array(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
#12.17040

ValueError: Error when checking : expected dense_input_19 to have shape (None, 55) but got array with shape (365, 56)

In [91]:
estimator.predict(np.array(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))

array([  50.96371841,   59.90620422,   54.8649826 ,   54.68158722,
         41.76868439,   40.57629013,   44.11871719,   45.60964966,
         49.65222168,   53.10652542,   45.69438171,   40.05634308,
         38.1469574 ,   37.70115662,   44.1137886 ,   55.5206337 ,
         49.42877197,   44.74610519,   41.30397797,   44.37857819,
         42.92516327,   50.06593323,   52.8028717 ,   60.01423264,
         51.20046997,   49.01172638,   51.47374344,   51.46464539,
         56.86006927,   59.14978027,   47.70895386,   38.38716125,
         34.326931  ,   35.56986237,   32.92583466,   34.94723129,
         41.38402557,   37.15632629,   34.258358  ,   33.69116211,
         32.24511719,   32.78614044,   38.40522385,   43.05294418,
         40.84880066,   35.10367966,   32.54026794,   31.8467598 ,
         33.73197937,   42.85925293,   49.46752548,   48.15474319,
         37.25727844,   38.22596359,   38.56145477,   40.16942978,
         49.19740295,   52.67054749,   52.5803299 ,   47.93579

In [85]:
estimator.predict(np.array(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","year"], axis=1)))

array([ 63.55615616,  61.68990326,  67.87526703,  54.31243515,
        43.10308838,  44.63131332,  44.45209122,  46.65771103,
        61.85708237,  59.97294998,  44.39570999,  40.64065933,
        41.09631729,  42.33072281,  50.37030029,  65.91745758,
        50.75778961,  39.18015671,  36.48463058,  40.38907242,
        41.10918045,  52.04777145,  63.60200119,  61.56138611,
        54.28500748,  45.63565063,  49.33920288,  53.82712173,
        67.46759033,  72.21656799,  53.96071243,  40.29394531,
        35.42073822,  35.03311539,  27.40891838,  30.89692879,
        45.79713058,  36.69977951,  29.26990318,  29.35329056,
        25.11638641,  26.30140686,  37.16840363,  52.05052567,
        46.58256912,  31.22219276,  23.68644524,  26.17887688,
        26.44341087,  37.41444016,  52.79363251,  45.55173111,
        41.39136124,  35.12322998,  38.86500168,  38.05392838,
        42.6211853 ,  49.78378677,  60.42580414,  49.01306534,
        47.63051987,  40.67090988,  28.03180122,  40.04

In [67]:
test_df["y"] = (y_rbf + pred_xgb)/2
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
#10.22758

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  se

In [76]:
test_df["y"] = (y_rbf + pred_xgb+forest_pred)/3
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)
#10.29414

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  se